### PROBLEM

Return to your Ames Data.  We have covered a lot of ground today, so let's summarize the things we could do to improve the performance of our original model that compared the Above Ground Living Area to the Logarithm of the Sale Price.
<div class="alert alert-info" role="alert">
1. Clean data, drop missing values (Done)
2. Transform data, code variables using either ordinal values or OneHotEncoder methods (Done)
3. Create more features from existing features (Done)
4. Split our data into testing and training sets
5. Normalize quantitative features
6. Use Regularized Regression methods and Polynomial regression to improve performance of model
</div>
Can you use some or all of these ideas to improve upon your initial model?

### Feature Engineering and Cleaning


We want to return to our Housing example and consider how to use some of `scikitlearn`'s functionality to deal with missing values.  We want to determine the correct way of dealing with these one by one, and use some of what we know about the data to inform these decisions.  If we have objects that are missing values, we can either exclude the observations, or encode the missing values using some kind of numerical value.  


In [324]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os as os
import scipy.stats as st
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

%matplotlib inline

ames = pd.read_csv('../data/ames_housing.csv')

ames.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [279]:
ames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [280]:
## Cleaning Data

# We need to clean the Ames Dataset columsn

ames['Alley'].value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [281]:
ames['Alley'] = ames['Alley'].fillna("None")

In [282]:
ames['Alley'].value_counts()

None    1369
Grvl      50
Pave      41
Name: Alley, dtype: int64

In [283]:
ames['FireplaceQu'].value_counts()

Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: FireplaceQu, dtype: int64

In [284]:
ames['FireplaceQu'] = ames['FireplaceQu'].fillna("None")

In [285]:
ames['MiscFeature'].value_counts()

Shed    49
Othr     2
Gar2     2
TenC     1
Name: MiscFeature, dtype: int64

In [286]:
ames['MoSold'].value_counts()

6     253
7     234
5     204
4     141
8     122
3     106
10     89
11     79
9      63
12     59
1      58
2      52
Name: MoSold, dtype: int64

Note the existence of a number of ordinal data points.  We can encode these to follow the data dictionary. https://ww2.amstat.org/publications/jse/v19n3/decock/datadocumentation.txt

In [287]:
ames = ames.replace({"BsmtCond": {"No": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}})

In [288]:
ames['BsmtCond'].value_counts()

3.0    1311
4.0      65
2.0      45
1.0       2
Name: BsmtCond, dtype: int64

In [289]:
ames = ames.replace({"BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})

In [290]:
ames['BsmtQual'].value_counts()

3.0    649
4.0    618
5.0    121
2.0     35
Name: BsmtQual, dtype: int64

**PROBLEMS**

Continue to code a few more columns and make sure to replace any `na` values in at least:

- `OverallQual` (Nothing done. Convert to 5 scale?)
- `OverallCond` (Nothing done. Convert to 5 scale?)
- `GarageQual` (DONE)
- `GarageCond` (DONE)
- `PoolArea` (DONE)
- `PoolQC` (DONE)

In [291]:
ames['PoolQC'].value_counts()

Gd    3
Ex    2
Fa    2
Name: PoolQC, dtype: int64

In [292]:
ames = ames.replace({"GarageQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})

In [293]:
ames['GarageQual'].value_counts()

3.0    1311
2.0      48
4.0      14
1.0       3
5.0       3
Name: GarageQual, dtype: int64

In [294]:
ames = ames.replace({"GarageCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})

In [295]:
ames['GarageCond'].value_counts()

3.0    1326
2.0      35
4.0       9
1.0       7
5.0       2
Name: GarageCond, dtype: int64

In [296]:
ames = ames.replace({"PoolArea" : {738 : 1, 648 : 1, 576: 1, 555 : 1, 519 : 1, 512 : 1, 480 : 1}})

In [297]:
ames['PoolArea'].value_counts()

0    1453
1       7
Name: PoolArea, dtype: int64

In [298]:
ames = ames.replace({"PoolQC" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})

ames["PoolQC"].fillna(0, inplace=True)

In [299]:
ames['PoolQC'].value_counts()

0.0    1453
4.0       3
2.0       2
5.0       2
Name: PoolQC, dtype: int64

**Deal With NaNs**

Replace any `na` values in the following columns if able:

- `LotFrontage` (Done)
- `MasVnrType` (Done)
- `MasVnrArea` (Done)
- `BsmtQual` (Done)
- `BsmtCond` (Done)
- `BsmtExposure` (Done)
- `BsmtFinType1` (Done)
- `BsmtFinType2` (Done)
- `Electrical` (Done)
- `GarageType` (Done)
- `GarageYrBlt` (Nothing done. Convert to 5 scale?)
- `GarageFinish` (Done)
- `Fence` (Done)
- `MiscFeature`

In [300]:
ames['LotFrontage'].value_counts()
ames["LotFrontage"].fillna(0, inplace=True)

In [301]:
ames['MasVnrType'].value_counts()
ames["MasVnrType"].fillna("None", inplace=True)

In [302]:
ames['MasVnrArea'].value_counts()
ames["MasVnrArea"].fillna(0, inplace=True)

In [303]:
ames['BsmtQual'].value_counts()
ames["BsmtQual"].fillna(0, inplace=True)
ames['BsmtQual'].value_counts()


3.0    649
4.0    618
5.0    121
0.0     37
2.0     35
Name: BsmtQual, dtype: int64

In [304]:
ames['BsmtCond'].value_counts()
ames["BsmtCond"].fillna(0, inplace=True)
ames['BsmtCond'].value_counts()

3.0    1311
4.0      65
2.0      45
0.0      37
1.0       2
Name: BsmtCond, dtype: int64

In [305]:
ames['BsmtExposure'].value_counts()
ames["BsmtExposure"].fillna(0, inplace=True)
ames = ames.replace({"BsmtExposure" : {"No" : 1, "Mn" : 2, "Av": 3, "Gd" : 4}})
ames['BsmtExposure'].value_counts()


1    953
3    221
4    134
2    114
0     38
Name: BsmtExposure, dtype: int64

In [306]:
ames['BsmtFinType1'].value_counts()
ames["BsmtFinType1"].fillna(0, inplace=True)
ames = ames.replace({"BsmtFinType1" : {"Unf" : 0, "LwQ" : 1, "BLQ": 2, "Rec" : 3, "ALQ" : 3, "GLQ" : 4}})
ames['BsmtFinType1'].value_counts()

0    467
4    418
3    353
2    148
1     74
Name: BsmtFinType1, dtype: int64

In [307]:
ames['BsmtFinType2'].value_counts()
ames["BsmtFinType2"].fillna(0, inplace=True)
ames = ames.replace({"BsmtFinType2" : {"Unf" : 0, "LwQ" : 1, "BLQ": 2, "Rec" : 3, "ALQ" : 3, "GLQ" : 4}})
ames['BsmtFinType2'].value_counts()

0    1294
3      73
1      46
2      33
4      14
Name: BsmtFinType2, dtype: int64

In [308]:
ames['Electrical'].value_counts()
ames["Electrical"].fillna(0, inplace=True)
ames = ames.replace({"Electrical" : {"Mix": 0, "FuseP" : 1, "FuseF" : 2, "FuseA" : 3, "SBrkr" : 4}})
ames['Electrical'].value_counts()


4    1334
3      94
2      27
1       3
0       2
Name: Electrical, dtype: int64

In [309]:
ames["GarageType"].value_counts()
ames["GarageType"].fillna(0, inplace=True)
ames = ames.replace({"GarageType" : {"Detchd" : 0, "CarPort" : 1, "Basment": 2, "Attchd" : 3, "2Types" : 4, "BuiltIn" : 4}})
ames["GarageType"].value_counts()


3    870
0    468
4     94
2     19
1      9
Name: GarageType, dtype: int64

In [310]:
ames["GarageFinish"].value_counts()
ames["GarageFinish"].fillna(0, inplace=True)
ames = ames.replace({"GarageFinish" : {"Unf" : 1, "RFn" : 2, "Fin": 3}})
ames["GarageFinish"].value_counts()


1    605
2    422
3    352
0     81
Name: GarageFinish, dtype: int64

In [311]:
ames["Fence"].value_counts()
ames["Fence"].fillna(0, inplace=True)
ames = ames.replace({"Fence" : {"MnWw" : 1, "MnPrv" : 2, "GdWo": 3, "GdPrv": 4}})
ames["Fence"].value_counts()



0    1179
2     157
4      59
3      54
1      11
Name: Fence, dtype: int64

In [312]:
ames["MiscFeature"].value_counts()
ames["MiscFeature"].fillna(0, inplace=True)
ames = ames.replace({"MiscFeature" : {"Othr" : 1, "Shed" : 2, "Gar2": 3, "TenC": 4}})       


In [313]:
ames = ames.replace({"BsmtQual" : {1: "isntgood"}})

In [314]:
ames["ExterQual"].value_counts()
ames["ExterCond"].value_counts()
ames = ames.replace({"ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex": 5}})       
ames = ames.replace({"ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex": 5}})       


In [315]:
ames["FireplaceQu"].value_counts()
ames = ames.replace({"FireplaceQu" : {"None" : 0, "Po" : 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}})   



In [316]:
ames["GarageCond"].fillna(0, inplace=True)
ames["GarageQual"].fillna(0, inplace=True)


### Adding New Features

We can create many new features to help improve our models performance.  For example, any of the measures that have multiple categories could be combined.  Take `Overall`, `Garage`, and `Pool` for example.  We can create combinations of the subcolumns as follows.

In [317]:
ames['BasementOverall'] = ames['BsmtCond'].astype('int') * ames['BsmtQual'].astype('int')
ames["GarageOverall"] = ames["GarageCond"].astype("int") * ames["GarageQual"].astype("int")
ames["PoolOverall"] = ames["PoolQC"].astype("int") * ames["PoolArea"].astype("int")
ames["OverallQualityScore"] = ames["OverallQual"] * ames["OverallCond"]
ames["ExternalOverall"] = ames["ExterQual"] * ames["ExterCond"]
ames["FireplaceOverall"] = ames["Fireplaces"] * ames["FireplaceQu"]

In [318]:
ames_numbers = ames.select_dtypes(include = "int64")
ames_numbers.columns


Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'ExterQual', 'ExterCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'Fence', 'MiscFeature', 'MiscVal', 'MoSold',
       'YrSold', 'SalePrice', 'BasementOverall', 'GarageOverall',
       'PoolOverall', 'OverallQualityScore', 'ExternalOverall',
       'FireplaceOverall'],
      dtype='object')

In [319]:
X = ames_numbers.drop("SalePrice", axis=1)
y = ames_numbers.SalePrice


In [320]:
# This creates a simple model based on all "int64" 
# columns from the dataset as the X variable and the Sale Price as Y variable

lr = LinearRegression()
lr.fit(X, y)

pred = lr.predict(X)

from sklearn.metrics import mean_squared_error

mse_1 = mean_squared_error(pred, y)

msre_1 = np.sqrt(mean_squared_error(pred, y))

print("Mean Squared Error is: {} and Mean Squared Root Error is: {}".format(mse_1, msre_1))

Mean Squared Error is: 1084237865.343484 and Mean Squared Root Error is: 32927.767390812936


In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(ames);

In [329]:
# This creates a heatmap to see what features appear to be highly correlated with Sales Price

ames_corr = ames.corr()
print(ames_corr)

                     Id  MSSubClass  LotFrontage   LotArea  OverallQual  \
Id             1.000000    0.011156    -0.010601 -0.033226    -0.028365   
MSSubClass     0.011156    1.000000    -0.386347 -0.139781     0.032628   
LotFrontage   -0.010601   -0.386347     1.000000  0.426095     0.251646   
LotArea       -0.033226   -0.139781     0.426095  1.000000     0.105806   
OverallQual   -0.028365    0.032628     0.251646  0.105806     1.000000   
OverallCond    0.012609   -0.059316    -0.059213 -0.005636    -0.091932   
YearBuilt     -0.012713    0.027850     0.123349  0.014228     0.572323   
YearRemodAdd  -0.021998    0.040581     0.088866  0.013788     0.550684   
MasVnrArea    -0.050298    0.022936     0.193458  0.104160     0.411876   
BsmtFinSF1    -0.005024   -0.069836     0.233633  0.214103     0.239666   
BsmtFinSF2    -0.005968   -0.065649     0.049900  0.111170    -0.059119   
BsmtUnfSF     -0.007940   -0.140759     0.132644 -0.002618     0.308159   
TotalBsmtSF   -0.015415  

# Correlation Findings

It appears the following factors are most correlated with SalesPrice:
* OverallQual - 0.79
* GarageCars - 0.64
* GarageArea - 0.62
* TotalBsmtSF - 0.61
* YearBuilt - 0.52
* YearRemodAdd - 0.50
* In addition, our calculated columns are included in our model


In [334]:
dropped_list = ['Id', 'MSSubClass', 'LotArea', 'OverallCond',
       'ExterQual', 'ExterCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageFinish',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'Fence', 'MiscFeature', 'MiscVal', 'MoSold',
       'YrSold', 'SalePrice']

X = ames_numbers.drop(dropped_list, axis=1)
y = ames_numbers.SalePrice


In [335]:
# This creates a simple model based on all "int64" 
# columns from the dataset as the X variable and the Sale Price as Y variable

lr = LinearRegression()
lr.fit(X, y)

pred = lr.predict(X)

from sklearn.metrics import mean_squared_error

mse_1 = mean_squared_error(pred, y)

msre_1 = np.sqrt(mean_squared_error(pred, y))

print("Mean Squared Error is: {} and Mean Squared Root Error is: {}".format(mse_1, msre_1))

Mean Squared Error is: 1613362551.3474014 and Mean Squared Root Error is: 40166.68459491524


# Results

* It appears our model has an MSRE currently \$40166 off from the actual sales price on average

## Next Steps / To Do:

1. Split data into testing and training sets
2. Normalize quantitative features (use scatter matrix or histogram to verify)
3. Use Polynomial regression to improve performance of model

